# Importando as bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pymannkendall import original_test
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm.auto import tqdm

from db import connection_db as conndb
from db import filters
from utils.estracao_interacao import ExtracaoInteracao

tqdm.pandas()

# Conectando ao banco de dados do MongoDB

Função para conectar ao MongoDB

In [ ]:
def conectar_mongodb(uri, db_name, collection_name):
    mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=collection_name)
    mongo_connection.connect()
    return mongo_connection.collection

In [ ]:
# Definindo os parâmetros de conexão
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
collection_name = 'dadosSemFiltros'
collection_name_filtered = 'posts7anos1anos'

collection = conectar_mongodb(uri, db_name, collection_name)
collection_filtrada = conectar_mongodb(uri, db_name, collection_name_filtered)

# Aplicando pipeline para preparação dos dados

Função para aplicar os pipelines de filtro

In [ ]:
def aplicar_pipelines(collection, collection_filtrada):
    collection_filters = filters.CollectionFilters(collection)
    collection_filters_likes = filters.CollectionFilters(collection_filtrada)

    # Aplicar pipelines
    collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
    collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')
    collection_filters.apply_pipeline3('postsComBDIAndInfos')
    collection_filters.apply_pipeline4('posts')
    collection_filters_likes.apply_pipeline6('likes')

    return collection_filters.collection, collection_filters_likes.collection

In [ ]:
# Aplicando os pipelinesc
collection, collection_filtrada = aplicar_pipelines(collection, collection_filtrada)

# Carregando os dados em um dataframe

In [ ]:
documentos = collection.find()
df_original = pd.DataFrame(list(documentos))

# Visualizar as primeiras linhas do dataframe
df_original.head()

In [ ]:
documentos_likes = collection_filtrada.find()
df_original_likes = pd.DataFrame(list(documentos_likes))

# Visualizar as primeiras linhas do dataframe
df_original_likes.head()

In [ ]:
# Transformando a coluna sexo em binária
df_original['sexo'] = df_original['sexo'].map({'F': 0, 'M': 1})

# Removendo colunas desnecessárias
df_original.drop(columns=['_id', 'diaDaSemana'], inplace=True)

# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida', 'choro',
    'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]


# Função para preencher valores nulos e converter o tipo de dado
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df

In [ ]:
df_original = preencher_e_converter(df_original, colunas_para_converter)

In [ ]:
# Remover usuarios que tem menos de 100 posts
df_original = df_original.groupby('id_usuario').filter(lambda value: len(value) >= 100)

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_posts(df):
    df['data'] = df['postCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]
    df_data = df_data[df_data['suicida'] == 3]

    # Remover posts vazios
    df_data = df_data.dropna(subset=['postMessage', 'postStory'], how='all')
    df_data = df_data[~((df_data['postMessage'].str.strip() == '') & (df_data['postStory'].str.strip() == ''))]

    return df_data

In [ ]:
df_posts = manipular_e_filtrar_posts(df_original)
df_posts.head()

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_likes(df):
    df['data'] = df['likeCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]

    return df_data

In [ ]:
df_likes = manipular_e_filtrar_likes(df_original_likes)

# Pegar os mesmos usuários do df_posts
df_likes = df_likes[df_likes['id_usuario'].isin(df_posts['id_usuario'])]
df_likes.head()

# Adicionar interações do usuário

In [ ]:
df_posts['quantPostMsg'] = df_posts['postMessage'].apply(
    lambda value: 1 if pd.notnull(value) and value.strip() != '' else 0)

df_posts['quantPostStory'] = df_posts['postStory'].apply(
    lambda value: 1 if pd.notnull(value) and value.strip() != '' else 0)

In [ ]:
extracao = ExtracaoInteracao(df_posts)
df_posts_interacoes = extracao.extract_interactions()

df_posts_interacoes.head()

Função para calcular e visualizar a distribuição de postagens por usuário

In [ ]:
def plotar_histogramas_posts(df):
    fig, axs = plt.subplots(3, 1, figsize=(10, 8), dpi=300)

    # Quantidade geral de postagens
    quant_posts = df.groupby('id_usuario').size().reset_index(name='quant_posts')
    axs[0].hist(quant_posts['quant_posts'], bins=50)
    axs[0].set_xlabel('Quant de Posts')
    axs[0].set_ylabel('Quant de Usuários')

    # Quantidade de postagens com mensagem
    quant_msg = df.groupby('id_usuario')['quantPostMsg'].sum().reset_index(name='quant_msg')
    axs[1].hist(quant_msg['quant_msg'], bins=50)
    axs[1].set_xlabel('Quant de Posts com Mensagem')
    axs[1].set_ylabel('Quant de Usuários')

    # Quantidade de postagens com story
    quant_story = df.groupby('id_usuario')['quantPostStory'].sum().reset_index(name='quant_story')
    axs[2].hist(quant_story['quant_story'], bins=50)
    axs[2].set_xlabel('Quant de Posts com Story')
    axs[2].set_ylabel('Quant de Usuários')

    plt.tight_layout()
    plt.show()

In [ ]:
plotar_histogramas_posts(df_posts_interacoes)

In [ ]:
def plotar_histogramas_interacoes(df):
    fig, axs = plt.subplots(3, 2, figsize=(10, 8), dpi=300)

    # Quantidade de atualização de Perfil
    quant_profile = df.groupby('id_usuario')['quantProfile'].sum().reset_index(name='quant_profile')
    axs[0, 0].hist(quant_profile['quant_profile'], bins=50)
    axs[0, 0].set_xlabel('Quant de Atualizações de Perfil')
    axs[0, 0].set_ylabel('Quant de Usuários')

    # Quantidade de atualização de Capa
    quant_cover = df.groupby('id_usuario')['quantCover'].sum().reset_index(name='quant_cover')
    axs[0, 1].hist(quant_cover['quant_cover'], bins=50)
    axs[0, 1].set_xlabel('Quant de Atualizações de Capa')
    axs[0, 1].set_ylabel('Quant de Usuários')

    # Quantidade de fotos com outras pessoas
    quant_photo_with_others = df.groupby('id_usuario')['quantAddPhotoWithOthers'].sum().reset_index(
        name='quant_photo_with_others')
    axs[1, 0].hist(quant_photo_with_others['quant_photo_with_others'], bins=50)
    axs[1, 0].set_xlabel('Quant de Fotos com Outras Pessoas')
    axs[1, 0].set_ylabel('Quant de Usuários')

    # Quantidade de está com outras pessoas
    quant_is_with_others = df.groupby('id_usuario')['quantIsWithOthers'].sum().reset_index(name='quant_is_with_others')
    axs[1, 1].hist(quant_is_with_others['quant_is_with_others'], bins=50)
    axs[1, 1].set_xlabel('Quant de Está com Outras Pessoas')
    axs[1, 1].set_ylabel('Quant de Usuários')

    # Quantidade de fotos adicionadas
    quant_add_photo = df.groupby('id_usuario')['quantAddPhoto'].sum().reset_index(name='quant_add_photo')
    axs[2, 0].hist(quant_add_photo['quant_add_photo'], bins=50)
    axs[2, 0].set_xlabel('Quant de Fotos Adicionadas')
    axs[2, 0].set_ylabel('Quant de Usuários')

    # Quantidade de fotos compartilhadas
    quant_shared_photo = df.groupby('id_usuario')['quantSharedPhoto'].sum().reset_index(name='quant_shared_photo')
    axs[2, 1].hist(quant_shared_photo['quant_shared_photo'], bins=50)
    axs[2, 1].set_xlabel('Quant de Fotos Compartilhadas')
    axs[2, 1].set_ylabel('Quant de Usuários')

    plt.tight_layout()
    plt.show()

In [ ]:
plotar_histogramas_interacoes(df_posts_interacoes)

In [ ]:
def plotar_histogramas_interacoes_2(df):
    fig, axs = plt.subplots(3, 2, figsize=(10, 8), dpi=300)

    # Quantidade de vídeos compartilhados
    quant_shared_video = df.groupby('id_usuario')['quantSharedVideo'].sum().reset_index(name='quant_shared_video')
    axs[0, 0].hist(quant_shared_video['quant_shared_video'], bins=50)
    axs[0, 0].set_xlabel('Quant de Vídeos Compartilhados')
    axs[0, 0].set_ylabel('Quant de Usuários')

    # Quantidade de links compartilhados
    quant_shared_link = df.groupby('id_usuario')['quantSharedLink'].sum().reset_index(name='quant_shared_link')
    axs[0, 1].hist(quant_shared_link['quant_shared_link'], bins=50)
    axs[0, 1].set_xlabel('Quant de Links Compartilhados')
    axs[0, 1].set_ylabel('Quant de Usuários')

    # Quantidade de posts compartilhados
    quant_shared_post = df.groupby('id_usuario')['quantSharedPost'].sum().reset_index(name='quant_shared_post')
    axs[1, 0].hist(quant_shared_post['quant_shared_post'], bins=50)
    axs[1, 0].set_xlabel('Quant de Posts Compartilhados')
    axs[1, 0].set_ylabel('Quant de Usuários')

    # Quantidade de eventos compartilhados
    quant_shared_event = df.groupby('id_usuario')['quantSharedEvent'].sum().reset_index(name='quant_shared_event')
    axs[1, 1].hist(quant_shared_event['quant_shared_event'], bins=50)
    axs[1, 1].set_xlabel('Quant de Eventos Compartilhados')
    axs[1, 1].set_ylabel('Quant de Usuários')

    # Quantidade de memórias compartilhadas
    quant_shared_memory = df.groupby('id_usuario')['quantSharedMemory'].sum().reset_index(name='quant_shared_memory')
    axs[2, 0].hist(quant_shared_memory['quant_shared_memory'], bins=50)
    axs[2, 0].set_xlabel('Quant de Memórias Compartilhadas')
    axs[2, 0].set_ylabel('Quant de Usuários')

    # Quantidade de status
    quant_status = df.groupby('id_usuario')['quantStatus'].sum().reset_index(name='quant_status')
    axs[2, 1].hist(quant_status['quant_status'], bins=50)
    axs[2, 1].set_xlabel('Quant de Atualizações de Status')
    axs[2, 1].set_ylabel('Quant de Usuários')

    plt.tight_layout()
    plt.show()

In [ ]:
plotar_histogramas_interacoes_2(df_posts_interacoes)

In [ ]:
df_agrupado_media = df_posts_interacoes.groupby(['id_usuario']).agg(
    mediaPostMsg=('quantPostMsg', lambda value: round(value.mean(), 4)),
    mediaProfile=('quantProfile', lambda value: round(value.mean(), 4)),
    mediaCover=('quantCover', lambda value: round(value.mean(), 4)),
    mediaAddPhotoWithOthers=('quantAddPhotoWithOthers', lambda value: round(value.mean(), 4)),
    mediaIsWithOthers=('quantIsWithOthers', lambda value: round(value.mean(), 4)),
    mediaAddPhoto=('quantAddPhoto', lambda value: round(value.mean(), 4)),
    mediaSharedPhoto=('quantSharedPhoto', lambda value: round(value.mean(), 4)),
    mediaSharedVideo=('quantSharedVideo', lambda value: round(value.mean(), 4)),
    mediaSharedLink=('quantSharedLink', lambda value: round(value.mean(), 4)),
    mediaSharedPost=('quantSharedPost', lambda value: round(value.mean(), 4)),
    mediaSharedEvent=('quantSharedEvent', lambda value: round(value.mean(), 4)),
    mediaSharedMemory=('quantSharedMemory', lambda value: round(value.mean(), 4)),
    mediaStatus=('quantStatus', lambda value: round(value.mean(), 4))
).reset_index()

In [ ]:
# Criar uma matriz de correlação

colunas_interacoes = df_agrupado_media[[
    'mediaPostMsg', 'mediaProfile', 'mediaCover', 'mediaAddPhotoWithOthers', 'mediaIsWithOthers',
    'mediaAddPhoto', 'mediaSharedPhoto', 'mediaSharedVideo', 'mediaSharedLink', 'mediaSharedPost',
    'mediaSharedEvent', 'mediaSharedMemory', 'mediaStatus'
]]

matriz_correlacao = colunas_interacoes.corr(method='spearman')

plt.figure(figsize=(12, 8))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação das Interações')
plt.show()

In [ ]:
from scipy.stats import spearmanr

# Testar para todas as combinações de variáveis
for i in range(len(colunas_interacoes.columns)):
    for j in range(i + 1, len(colunas_interacoes.columns)):
        x = colunas_interacoes.iloc[:, i]
        y = colunas_interacoes.iloc[:, j]

        spearman_corr, p_value = spearmanr(x, y)
        print(
            f'Correlação de Spearman entre {colunas_interacoes.columns[i]} e {colunas_interacoes.columns[j]}: {spearman_corr}')
        print(f'P-valor de Spearman: {p_value}')

In [ ]:
df_agrupado_media['mediaPostMsgWithOthers'] = df_agrupado_media[[
    'mediaPostMsg', 'mediaAddPhotoWithOthers', 'mediaIsWithOthers'
]].mean(axis=1).round(4)

df_agrupado_media['mediaAtualizacaoPerfil'] = df_agrupado_media[[
    'mediaProfile', 'mediaCover', 'mediaStatus'
]].mean(axis=1).round(4)

df_agrupado_media['mediaCompartilhamento'] = df_agrupado_media[[
    'mediaSharedPhoto', 'mediaSharedVideo', 'mediaSharedLink',
    'mediaSharedPost', 'mediaSharedEvent', 'mediaSharedMemory'
]].mean(axis=1).round(4)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

variaveis = df_agrupado_media[[
    'mediaPostMsgWithOthers', 'mediaAtualizacaoPerfil', 'mediaCompartilhamento', 'mediaAddPhoto'
]]

# Normalizar as variáveis
scaler = StandardScaler()
variaveis_normalizadas = scaler.fit_transform(variaveis)

In [ ]:
# Aplicar PCA para reduzir a dimensionalidade
pca = PCA(n_components=2)
pca_resultado = pca.fit_transform(variaveis_normalizadas)

# Ver a variância explicada por cada componente
print(pca.explained_variance_ratio_)

In [ ]:
from sklearn.metrics import davies_bouldin_score

k_values = range(2, 11)
silhouette_scores = []
davies_bouldin_scores = []

for k in k_values:
    # Aplicar o K-Means com k clusters
    kmeans = KMeans(n_clusters=k, random_state=42)

    # Ajustar o modelo nos dados normalizados
    kmeans.fit(variaveis_normalizadas)

    # Obter os rótulos dos clusters
    labels = kmeans.labels_

    # Calcular o Coeficiente de Silhueta
    silhouette_avg = silhouette_score(variaveis_normalizadas, labels)
    silhouette_scores.append(silhouette_avg)

    # Calcular o Índice de Davies-Bouldin
    db_index = davies_bouldin_score(variaveis_normalizadas, labels)
    davies_bouldin_scores.append(db_index)

In [ ]:
# Plotar os resultados
fig, ax1 = plt.subplots(figsize=(12, 8))

# Gráfico do Coeficiente de Silhueta
ax1.plot(k_values, silhouette_scores, marker='o', label='Coeficiente de Silhueta')
ax1.set_xlabel('Número de Clusters (k)')
ax1.set_ylabel('Coeficiente de Silhueta')
ax1.set_title('Avaliação do K-Means com Coeficiente de Silhueta e Índice de Davies-Bouldin')
ax1.legend(loc='upper left')

# Gráfico do Índice de Davies-Bouldin
ax2 = ax1.twinx()
ax2.plot(k_values, davies_bouldin_scores, marker='s', color='red', label='Índice de Davies-Bouldin')
ax2.set_ylabel('Índice de Davies-Bouldin')
ax2.legend(loc='upper right')
plt.show()

In [ ]:
# Printar o silhouette score junto com o número de clusters
for k, score in zip(k_values, silhouette_scores):
    print(f'Número de Clusters: {k} - Coeficiente de Silhueta: {score}')

print('\n')

# Printar o índice de Davies-Bouldin junto com o número de clusters
for k, score in zip(k_values, davies_bouldin_scores):
    print(f'Número de Clusters: {k} - Índice de Davies-Bouldin: {score}')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)

kmeans.fit(variaveis_normalizadas)

df_agrupado_media['cluster'] = kmeans.labels_

print(df_agrupado_media[['id_usuario', 'cluster']].head())

In [ ]:
df_agrupado_media['cluster'].value_counts()

In [ ]:
def teste_mk(df_cluster, periodo, tipo_serie='posts'):
    resultados_mk = []

    for usuario in df_cluster['id_usuario'].unique():
        df_usuario = df_cluster[df_cluster['id_usuario'] == usuario]

        eventos_por_dia = df_usuario.groupby('data').size()

        if len(eventos_por_dia) >= 2 * periodo:
            try:
                resultado = seasonal_decompose(eventos_por_dia, model='additive', period=periodo)
                tendencia = resultado.trend.dropna()

                if len(tendencia) > 0:
                    result = original_test(tendencia)

                    if hasattr(result, 'trend'):
                        resultados_mk.append({
                            'id_usuario': usuario,
                            'trend': result.trend,
                            'h': result.h,
                            'p-valor': result.p,
                            'z': result.z,
                            'Tau': result.Tau,
                            's': result.s,
                            'var_s': result.var_s,
                            'slope': result.slope,
                            'intercept': result.intercept
                        })
                    else:
                        print(f"Usuário {usuario} - Resultado do teste de Mann-Kendall sem 'trend'.")
                else:
                    print(f"Usuário {usuario} - Não foi possível decompor dados suficientes para tendência.")
            except ValueError as e:
                print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
        else:
            print(
                f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(eventos_por_dia)}")

    # Criando o DataFrame apenas se houver resultados suficientes
    if resultados_mk:
        df_resultados_mk = pd.DataFrame(resultados_mk)

        if 'trend' in df_resultados_mk.columns:
            contagem_tendencias = df_resultados_mk['trend'].value_counts()
            print(contagem_tendencias)

            total_usuarios = df_resultados_mk.shape[0]
            percentuais = (contagem_tendencias / total_usuarios) * 100
            print(percentuais)

            # Gráfico de barras das tendências
            plt.figure(figsize=(8, 6))
            sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
            plt.title(f'Distribuição de Tendências entre Usuários ({tipo_serie.capitalize()})')
            plt.xlabel('Tipo de Tendência')
            plt.ylabel('Número de Usuários')
            plt.show()

            # Gráfico de distribuição dos slopes
            plt.figure(figsize=(10, 6))
            sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
            plt.title(f'Distribuição dos Slopes entre Usuários ({tipo_serie.capitalize()})')
            plt.xlabel('Slope')
            plt.ylabel('Frequência')
            plt.show()

        else:
            print("Nenhuma tendência foi detectada nos dados.")

        return df_resultados_mk
    else:
        print("Nenhum dado disponível para análise.")
        return pd.DataFrame()

In [ ]:
# buscar no dataframe df_posts_interacoes os usuários do cluster 0, 1 e 2
df_cluster0_media = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(df_agrupado_media[df_agrupado_media['cluster'] == 0]['id_usuario'])]
df_cluster1_media = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(df_agrupado_media[df_agrupado_media['cluster'] == 1]['id_usuario'])]
df_cluster2_media = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(df_agrupado_media[df_agrupado_media['cluster'] == 2]['id_usuario'])]

In [ ]:
resultados_mk_cluster0 = teste_mk(df_cluster0_media, 30)
resultados_mk_cluster1 = teste_mk(df_cluster1_media, 30)
resultados_mk_cluster2 = teste_mk(df_cluster2_media, 30)